## C S 329E HW 5

# Crossvalidation and hyperparameter selection

## Elena Zhang and Emily Zhou

For this week's homework we are going to explore the cross validation testing methodology and applying that to get error estimates on the two algorithms we've used so far:
  - Linear Regression
  - Decision Tree classification
  

In [1]:
# import the libraries! If you want to add things here for visualization, please do, 
# but only use sklearn when prompted
import pandas as pd
import numpy as np
from sklearn import tree 
from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

# Part 1 - Calculate Generalized Error on Linear Regression with k-fold Cross Validation

## Q1.1 Load in the diabetes data set as a pandas dataframe and series.  
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html).  Name your features dataframe (the independent variables) `df_X` and your target value (the dependent variable) series `s_y`

In [2]:
#load data
diabetes = load_diabetes()
features = diabetes.feature_names #get feature names

#create s_y - class label
s_y = diabetes.target

#create df_X - attributes
df_X = pd.DataFrame(diabetes.data, columns = features)

## Q1.2 Define a function that creates a linear least squares regression model 
This function should take in two parameters, `df_X`, and `s_y` and return the least squares regression model, $\hat{\beta}$ (using the notation from the ESLII book chapter 3 and HW3).  You can not use any libraries outside of pandas and numpy. Note that the length of beta_hat should be the number of columns in `df_X` + 1. 

In [3]:
def get_linear_regression_model( df_X, s_y ):
    # your code here
    X = pd.concat([pd.DataFrame({'intercept':np.ones(len(df_X))}), df_X], axis=1)
    (beta_hat, residuals, rank, s) = np.linalg.lstsq(X, s_y, rcond=-1)
    return beta_hat

In [4]:
# code to check beta_hat
np.random.seed(23)
beta_hat = get_linear_regression_model( pd.DataFrame(np.random.random((34,4))), pd.Series(np.random.random(34)*10.0) )
beta_hat

array([ 4.18818425,  1.77890808,  0.74032569, -1.3506416 ,  0.14535984])

## Q1.3 Define a function that partitions the dataframe and series data into dictionaries
This function should take in three parameters, `df_X`, `s_y`, and `k`, and returns a tuple of two dictionaries.
In both dictionaries the key is the `k` value (an integer from one to `k` inclusive).
The first dictionary will have the dataframe of the training data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable).
The second dictionary will have the series of the target data that contains approximately $\frac{1}{k}$ of the data (variation due to randomness is acceptable). Please note the targets _must match_ the same rows as the dataframe at this index, e.g, the length of the kth partition is the same for the dataframe and series.

Call that function with $k=5$ and create the dictionaries `dict_k_df_X` and `dict_k_s_y`. Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Here is some example output from checking the length of the folds:
```
Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 96 and length of series is 96
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 79 and length of series is 79
Fold 5 length of dataframe is 91 and length of series is 91
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df
```

In [5]:
def partition_data( df_X, s_y, k ):
    # your code here
    #get partition sizes using integer division
    sizes = []
    length = len(df_X)
    size = length // k
    remaining = length - (size * k)
    sizes += [size] * (k-1)
    sizes.append(size + remaining)
    
    #shuffle data set indices
    indices = np.arange(0,len(df_X))
    np.random.shuffle(indices)
    
    #intialize the dictionaries
    dict_k_df_X = {}
    dict_k_s_y = {}
    start = 0
    
    #fill in the dictionaries
    for i in range(len(sizes)):
        
        #grab index numbers for each fold
        fold_indices = indices[start:start+sizes[i]]
        
        #grab df_X rows
        fold_rows = df_X.iloc[fold_indices]                   
        #put df_X rows in dict_k_df_X
        dict_k_df_X[i + 1] = fold_rows
        
        #grab target rows
        target_rows = pd.DataFrame(s_y).iloc[fold_indices]                     
        #put target rows in dict_k_s_y
        dict_k_s_y[i + 1] = target_rows
        
        #update next partition index start
        start += sizes[i]
        
    return(dict_k_df_X,dict_k_s_y)
        
        
#     COMMENTS FOR TESTING        
#     for key,value in dict_k_df_X.items():
#         print(key)
#         print(value)

#     for key,value in dict_k_s_y.items():
#         print(key)
#         print(value)

#     for i in range(1,6):
#         print(dict_k_df_X[i])
#         print(dict_k_s_y[i])
        

In [6]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

In [7]:
# Check fold sizes
total_rows = 0
for key in dict_k_df_X.keys():
    print('Fold',key,'length of dataframe is',len(dict_k_df_X[key]),'and length of series is',len(dict_k_s_y[key])) 
    total_rows += len(dict_k_df_X[key])
print('The sum of the number of elements in each fold is',total_rows,'and there are',len(df_X),'rows in the original df')

Fold 1 length of dataframe is 88 and length of series is 88
Fold 2 length of dataframe is 88 and length of series is 88
Fold 3 length of dataframe is 88 and length of series is 88
Fold 4 length of dataframe is 88 and length of series is 88
Fold 5 length of dataframe is 90 and length of series is 90
The sum of the number of elements in each fold is 442 and there are 442 rows in the original df


## Q1.4 Define a function that calculates a regression metric
This function should accept two series of equal length $n$ numpy arrays, `s_y`, and `s_y_hat`. The metric it should calculate is the mean absolute error, $MAE = \sum\limits_{i=1}^n\frac{|{s\_y_i - {s\_y\_hat}_i}|}{n}$ 

Test your function by using the vectors:
```
x = np.array([1,2,3])
y = np.array([2,2,3])
```


In [8]:
def get_mae( s_y, s_y_hat):
    # your code here
    mae = 0
    length = len(s_y)
    for i in range(length):
        num = abs(s_y[i]-s_y_hat[i]) / length
        mae += num
    return mae

In [9]:
# Test it 
x = np.array([1,2,3])
y = np.array([2,2,3])
get_mae(x,y)

0.3333333333333333

## Q1.5 Calculate the $MAE$ for each fold
For each fold in your dictionaries, calculate the $MAE$.  Use the partition number key as the test set, and all other partitions as the train set. 

Print the min, max, and mean $MAE$ of your 5 folds. 

In [10]:
mae = np.array([])
for k in dict_k_df_X.keys():
    
    # your code here
    #get test data
    test_df_X = pd.concat([pd.DataFrame({'intercept':np.ones(len(dict_k_df_X[k]))}), dict_k_df_X[k].reset_index(drop=True)], axis=1)
    s_y = dict_k_s_y[k].reset_index(drop=True).to_numpy()
    
    #get training data
    filtered_df_X = {key:value for key,value in dict_k_df_X.items() if key != k}
    train_df_X = pd.concat(filtered_df_X.values()).reset_index(drop=True)
    
    #print(train_df_X)
    filtered_s_y = {key:value for key,value in dict_k_s_y.items() if key != k}
    train_s_y = pd.concat(filtered_s_y.values()).reset_index(drop=True)
    
    #create regression model with training data
    beta_hat = get_linear_regression_model(train_df_X,train_s_y)
    
    #predict on test data
    s_y_hat = np.matmul(test_df_X,beta_hat).to_numpy()
    
    #calculate MAE
    mae = np.append(mae, get_mae(s_y,s_y_hat) )   

In [11]:
print("The min MAE is {:.2f}, the max MAE is {:.2f}, and the mean MAE is {:.2f}".format(mae.min(),mae.max(),mae.mean()))

The min MAE is 42.27, the max MAE is 46.97, and the mean MAE is 44.40


# Part 2 - Find the best hyperparameter to use in a Decision Tree 

## Q2.1 Load the iris data in as a pandas dataframe and a series
Documentation on the data set is [here](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html). Name your features dataframe (the independent variables) `df_X` and your class label (the dependent variable) series `s_y`

In [12]:
#load data
iris = load_iris()
features = iris.feature_names #get feature names

#create s_y - class label
s_y = iris.target

#create df_X - attributes
df_X = pd.DataFrame(iris.data, columns = features)

## Q2.2 Partition `df_X` and `s_y` into $5$ partitions of roughly equal size
Make 2 dictionaries, with the key of each dictionary the fold number.  The value of the dictionary `dict_k_df_X` is the $k^{th}$ partition of the data, and the value of the dictionary `dict_k_s_y` is the corresponding $k^{th}$ target classification.  Print out the number of rows in each fold.  Check that the number of data points in each partition totals the number of data points in the entire dataset. 

Note, you can reuse the functions from Section 1. 

In [13]:
(dict_k_df_X, dict_k_s_y) = partition_data( df_X, s_y, 5 )

# Check fold sizes
total_rows = 0
for key in dict_k_df_X.keys():
    print('Fold',key,'length of dataframe is',len(dict_k_df_X[key]),'and length of series is',len(dict_k_s_y[key])) 
    total_rows += len(dict_k_df_X[key])
print('The sum of the number of elements in each fold is',total_rows,'and there are',len(df_X),'rows in the original df')

Fold 1 length of dataframe is 30 and length of series is 30
Fold 2 length of dataframe is 30 and length of series is 30
Fold 3 length of dataframe is 30 and length of series is 30
Fold 4 length of dataframe is 30 and length of series is 30
Fold 5 length of dataframe is 30 and length of series is 30
The sum of the number of elements in each fold is 150 and there are 150 rows in the original df


## Q2.3 Define a function that calculates accuracy
The function should accept two series and compare each element for equality.  The accuracy is the number of equal elements divided by the total number of elements.

Test your accuracy function by calling it with the `s_y` loaded from the iris data set and an array of the same length containing all $1$ values. 

In [14]:
def get_acc( s_1, s_2 ):
    # your code here
    return sum(s_1 == s_2) / len(s_1)
    

In [15]:
get_acc(s_y,np.ones(len(s_y)))

0.3333333333333333

## Q2.4 Using Nested Cross validation, find the best hyperparameter
Use the [Decision Tree Classifier](https://scikit-learn.org/stable/modules/tree.html#classification) class to build a decision tree inside of a 5-fold cross validation loop.  The partitions you already created in 2.2 will be the outer loop.  In the inside loop you should use 4-fold cross validation (so you don't have to partition _again_) to find the best value for `min_impurity_decrease`.  Use the Gini Index as your impurity measure. 
    Calculate the mean accuracy across the 4 folds of your inner loop for all the candidate `min_impurity_decrease` values, and print the value.  Use the array `np.array([0.1,0.25,0.3,0.4])` as the candidates for the best hyperparameter. If there is a tie (two `min_impurity_decrease` values give the same highest accuracy), choose the lowest `min_impurity_decrease` value. 

For each inner loop, select the best `min_impurity_decrease` and train the outer fold training data on using that value. 

For each of the 5 executions of the inner loop, your output should look something like this:
```
Testing 0.10 min impurity decrease
	Average accuracy over 4 folds is 0.95
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.86
Testing 0.30 min impurity decrease
	Average accuracy over 4 folds is 0.63
Testing 0.40 min impurity decrease
	Average accuracy over 4 folds is 0.27

Best min impurity decrease is 0.1

```

In [16]:
possible_min_impurity_decrease = np.array([0.1,0.25,0.3,0.4])

# Outer loop
outer_acc = np.array([])
for k in dict_k_df_X.keys():
     # your code here
    avg_acc = np.array([])

    #get outer df_X data
    outer_test = dict_k_df_X[k]
    filtered_outer_df_X_train = {key:value for key,value in dict_k_df_X.items() if key != k}
    outer_train = pd.concat(filtered_outer_df_X_train.values()).reset_index(drop = True)
    
    #get outer s_y data
    s_y_test = dict_k_s_y[k][0].to_numpy()
    filtered_s_y_train = {key:value for key, value in dict_k_s_y.items() if key != k}
    s_y_train = pd.concat(filtered_s_y_train.values()).reset_index(drop = True)
    
    for pos_min_impurity in possible_min_impurity_decrease:
        
        inner_acc = np.array([])
        # Inner loop cross validation code here (use 4 folds, where the fold does not include k)
        for test_k in filtered_outer_df_X_train.keys():
            
            #get inner df_X data
            inner_test = filtered_outer_df_X_train[test_k]
            filtered_inner_train = {key:value for key, value in filtered_outer_df_X_train.items() if key != test_k}
            inner_train = pd.concat(filtered_inner_train.values()).reset_index(drop = True)

            #get inner s_y data
            inner_s_y_test = filtered_s_y_train[test_k][0].to_numpy()
            filtered_inner_s_y_train = {key:value for key, value in filtered_s_y_train.items() if key != test_k}
            inner_s_y_train = pd.concat(filtered_inner_s_y_train.values()).reset_index(drop = True)
            
            #build model
            clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = pos_min_impurity)
            clf = clf.fit(inner_train, inner_s_y_train)
            
            #predict and evaluate model accuracy
            pred = clf.predict(inner_test)
            acc = get_acc(pred, inner_s_y_test)
            
            inner_acc = np.append(inner_acc, acc)
        
        #find mean accuracy across 4 folds
        avg_inner_acc = np.average(inner_acc)
        avg_acc = np.append(avg_acc, avg_inner_acc)
        
        print("Testing " + str(pos_min_impurity) + " min impurity decrease")
        print("\tAverage accuracy over 4 folds is " + str(avg_inner_acc))
        

    # get index/indices of highest accuracy
    ind = np.where(avg_acc == max(avg_acc))    
    # get hyperparameter corresponding to index; if tie use smallest impurity
    best_min_imp = min(possible_min_impurity_decrease[ind])
    print("Best min impurity decrease is " + str(best_min_imp))
    
    # Use best min impurity decrease to train model
    outer_clf = tree.DecisionTreeClassifier(criterion = 'gini', min_impurity_decrease = best_min_imp)
    outer_clf = outer_clf.fit(outer_train, s_y_train)
    
    #predict using test data
    outer_pred = outer_clf.predict(outer_test)
    
    # outer accuracy calculation 
    this_acc = get_acc(outer_pred, s_y_test)
    outer_acc = np.append(outer_acc,this_acc) # make sure and calculate this_acc in your loop
    
    print()


Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.9416666666666667
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.8416666666666666
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.6333333333333333
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.2833333333333333
Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.9500000000000001
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.9500000000000001
Testing 0.3 min impurity decrease
	Average accuracy over 4 folds is 0.6416666666666666
Testing 0.4 min impurity decrease
	Average accuracy over 4 folds is 0.27499999999999997
Best min impurity decrease is 0.1

Testing 0.1 min impurity decrease
	Average accuracy over 4 folds is 0.9333333333333333
Testing 0.25 min impurity decrease
	Average accuracy over 4 folds is 0.9333333333333333
Testing 0.3 min impurity decrease
	Average accuracy over

## Q2.5 Show the generalized performance of the classifier 
Show the generalized performance of the classifier by printing the min, max, and mean accuracy of the outer fold test sets.

In [17]:
#min accuracy
min(outer_acc)

0.8666666666666667

In [18]:
#max accuracy
max(outer_acc)

0.9666666666666667

In [19]:
#mean accuracy
np.average(outer_acc)

0.9333333333333333